In [67]:
gg.y.shape

(170, 1)

In [70]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(y,gg.y)

/home/rr/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=1, p=2, weights='uniform')

In [132]:
y2,y

(array([[ 0.1298492 ,  0.13069262,  0.12919304,  0.12933622,  0.12987847,
          0.12980035,  0.13000751,  0.1295728 ,  0.13025351,  0.13090743,
          0.12947404,  0.12958416,  0.12787188,  0.13091456,  0.13185618,
          0.13064505,  0.12947454,  0.13119055,  0.12970226,  0.13082989]]),
 array([[ 0.12962439,  0.13053459,  0.12901379,  0.1290788 ,  0.12976534,
          0.12959154,  0.12979294,  0.12944969,  0.13009259,  0.1307194 ,
          0.1292543 ,  0.12944066,  0.12771392,  0.13069144,  0.13171217,
          0.13044065,  0.12931946,  0.13096522,  0.12955307,  0.13063631]]))

In [37]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.imshow(testset[2].X[0:1,:].reshape(100,100),cmap=cm.gray)
plt.show()

In [42]:
plt.imshow(y.reshape(100,100),cmap=cm.gray_r)
plt.show()
plt.figure()

In [41]:
y = f(testset[2].X[0:1,:])

In [43]:
model = encoder_models[1]
X = model.get_input_space().make_theano_batch(batch_size=170)
Y = model.reconstruct( X )
f = theano.function( [X], Y )

In [10]:
from pylearn2.utils import serial
model_path = "my_model.pkl"
model = serial.load( model_path )
X = model.get_input_space().make_theano_batch(batch_size=1)
Y = model.reconstruct( X )
f = theano.function( [X], Y )

KeyboardInterrupt: (None, <function _reconstructor at 0x7f6feada6aa0>, (<class 'theano.tensor.type.TensorType'>, <type 'object'>, None))

In [16]:
X = encoder_models[0].get_input_space().make_theano_batch(batch_size=1)
Y = encoder_models[0].reconstruct( X )
f = theano.function( [X], Y )

In [58]:
from pylearn2.utils import serial
model_path = "my_model.pkl"
model = serial.load( model_path )
X = model.get_input_space().make_theano_batch(batch_size=170)
Y = model.encode( X )
f = theano.function( [X], Y )

In [46]:
from pylearn2.models.model import Model
from pylearn2.space import VectorSpace
from pylearn2.models.mlp import PretrainedLayer
import theano
from theano import tensor
from pylearn2.utils import sharedX
import theano.tensor as T

from __future__ import print_function

__author__ = "Li Yao"
"""
See readme.txt

A small example of how to glue shining features of pylearn2 together
to train models layer by layer.
"""

MAX_EPOCHS_UNSUPERVISED = 1
MAX_EPOCHS_SUPERVISED = 2

from pylearn2.config import yaml_parse
from pylearn2.corruption import BinomialCorruptor
from pylearn2.corruption import GaussianCorruptor
from pylearn2.costs.mlp import Default
from pylearn2.models.autoencoder import Autoencoder, DenoisingAutoencoder
from pylearn2.models.rbm import GaussianBinaryRBM
from pylearn2.models.DRM import DeepEncoder
from pylearn2.models.softmax_regression import SoftmaxRegression
from pylearn2.training_algorithms.sgd import SGD
from pylearn2.costs.autoencoder import MeanSquaredReconstructionError
from pylearn2.termination_criteria import EpochCounter
from pylearn2.datasets.dense_design_matrix import DenseDesignMatrix
from pylearn2.energy_functions.rbm_energy import GRBM_Type_1
from pylearn2.blocks import StackedBlocks
from pylearn2.datasets.transformer_dataset import TransformerDataset
from pylearn2.costs.ebm_estimation import SMD
from pylearn2.costs.DRM import DRMCost
from pylearn2.training_algorithms.sgd import MonitorBasedLRAdjuster
from pylearn2.train import Train
from optparse import OptionParser
from pylearn2.datasets.c3 import load_data

import numpy

MAX_EPOCHS_UNSUPERVISED = 1
MAX_EPOCHS_SUPERVISED = 2


class ToyDataset(DenseDesignMatrix):
    def __init__(self):
        # simulated random dataset
        rng = numpy.random.RandomState(seed=42)
        data = rng.normal(size=(1000, 10))
        self.y = numpy.random.binomial(1, 0.5, (1000, 1))
        super(ToyDataset, self).__init__(X=data, y=self.y, y_labels=2)

def get_dataset_toy():
    """
    The toy dataset is only meant to used for testing pipelines.
    Do not try to visualize weights on it. It is not picture and
    has no color channel info to support visualization
    """
    trainset = ToyDataset()
    testset = ToyDataset()

    return trainset, testset

def get_dataset_cifar10():

    print('loading CIFAR-10 dataset...')

    # We create the dataset by parsing YAML strings describing the dataset.
    # The yaml parser will automatically tag trainset and testset with a
    # yaml_src field containing the YAML string that was used to specify them.
    # This is useful because later the training algorithm can store this YAML
    # string in the saved model to efficiently describe exactly what data it
    # was trained on.
    template = \
"""!obj:pylearn2.datasets.cifar10.CIFAR10 {
which_set: %s,
center: 1,
rescale: 1,
}"""
    trainset = yaml_parse.load(template % "train")
    testset = yaml_parse.load(template % "test")

    print('...done loading CIFAR-10.')

    return trainset, testset

def get_dataset_mnist():

    print('loading MNIST dataset...')

    # We create the dataset by parsing YAML strings describing the dataset.
    # The yaml parser will automatically tag trainset and testset with a
    # yaml_src field containing the YAML string that was used to specify them.
    # This is useful because later the training algorithm can store this YAML
    # string in the saved model to efficiently describe exactly what data it
    # was trained on.
    template = \
"""!obj:pylearn2.datasets.mnist.MNIST {
which_set: %s,
}"""
    trainset = yaml_parse.load(template % "train")
    testset = yaml_parse.load(template % "test")

    print('...done loading MNIST.')

    return trainset, testset

def get_autoencoder(structure):
    n_input, n_output = structure
    config = {
        'nhid': n_output,
        'nvis': n_input,
        'tied_weights': True,
        'act_enc': 'sigmoid',
        'act_dec': 'sigmoid',
        'irange': 0.001,
    }
    return Autoencoder(**config)

def get_denoising_autoencoder(structure):
    n_input, n_output = structure
    curruptor = BinomialCorruptor(corruption_level=0.5)
    config = {
        'corruptor': curruptor,
        'nhid': n_output,
        'nvis': n_input,
        'tied_weights': True,
        'act_enc': 'sigmoid',
        'act_dec': 'sigmoid',
        'irange': 0.001,
    }
    return DenoisingAutoencoder(**config)

def get_grbm(structure):
    n_input, n_output = structure
    config = {
        'nvis': n_input,
        'nhid': n_output,
        "irange" : 0.05,
        "energy_function_class" : GRBM_Type_1,
        "learn_sigma" : True,
        "init_sigma" : .4,
        "init_bias_hid" : -2.,
        "mean_vis" : False,
        "sigma_lr_scale" : 1e-3
        }

    return GaussianBinaryRBM(**config)

def get_logistic_regressor(structure):
    n_input, n_output = structure

    layer = SoftmaxRegression(n_classes=n_output, irange=0.02, nvis=n_input)

    return layer

def get_layer_trainer_logistic(layer, trainset):
    # configs on sgd

    config = {'learning_rate': 0.1,
              'cost' : Default(),
              'batch_size': 10,
              'monitoring_batches': 10,
              'monitoring_dataset': trainset,
              'termination_criterion': EpochCounter(max_epochs=MAX_EPOCHS_SUPERVISED),
              'update_callbacks': None
              }

    train_algo = SGD(**config)
    model = layer
    return Train(model = model,
            dataset = trainset,
            algorithm = train_algo,
            extensions = None)

def get_layer_trainer_sgd_autoencoder(layer, trainset):
    # configs on sgd
    train_algo = SGD(
            learning_rate = 0.1,
              cost =  DRMCost(),
              batch_size =  10,
              monitoring_batches = 10,
              monitoring_dataset =  trainset,
              termination_criterion = EpochCounter(max_epochs=10),
              update_callbacks =  None
              )

    model = layer
    extensions = None
    return Train(model = model,
            algorithm = train_algo,
            extensions = extensions,
            save_path='my_model.pkl'
            ,save_freq=1,
            dataset = trainset)

def get_layer_trainer_sgd_rbm(layer, trainset,epochs):
    train_algo = SGD(
        learning_rate = 1e-1,
        batch_size =  5,
        #"batches_per_iter" : 2000,
        monitoring_batches =  20,
        monitoring_dataset =  trainset,
        cost = SMD(corruptor=GaussianCorruptor(stdev=0.4)),
        termination_criterion =  EpochCounter(max_epochs=epochs),
        )
    model = layer
    extensions = [MonitorBasedLRAdjuster()]
    return Train(model = model, algorithm = train_algo,
                 save_path='grbm.pkl',save_freq=1,
                 extensions = extensions, dataset = trainset)


    
def maind(args=None):
    #trainset, testset, = get_dataset_mnist()
    trainset, testset = load_data()
    n_output = 10
    
    design_matrix = trainset.get_design_matrix()
    n_input = design_matrix.shape[1]

    # build layers
    layers = []
    structure = [[n_input, 50], [50, 10], [10, 5],[5, n_input]]
    # layer 0: gaussianRBM
    layers.append(get_grbm(structure[0]))
    # layer 1: denoising AE
    layers.append(get_grbm(structure[1]))
    # layer 2: AE
    layers.append(get_grbm(structure[2]))

    # layer 3: logistic regression used in supervised training
    #layers.append(get_logistic_regressor(structure[3]))

    # construct training sets for different layers
    trainset = [ trainset ,
                TransformerDataset( raw = trainset, transformer = layers[0] ),
                TransformerDataset( raw = trainset, transformer = StackedBlocks( layers[0:2] )),
                TransformerDataset( raw = trainset, transformer = StackedBlocks( layers[0:3] ))  
                ]
    # construct layer trainers
    layer_trainers = []
    layer_trainers.append(get_layer_trainer_sgd_rbm(layers[0], trainset[0],20))
    layer_trainers.append(get_layer_trainer_sgd_rbm(layers[1], trainset[1],5))
    layer_trainers.append(get_layer_trainer_sgd_rbm(layers[2], trainset[2],5))
    #layer_trainers.append(get_layer_trainer_logistic(layers[3], trainset[3]))


    
    # unsupervised pretraining
    for i, layer_trainer in enumerate(layer_trainers[0:3]):
        print('-----------------------------------')
        print(' Unsupervised training layer %d, %s'%(i, layers[i].__class__))
        print('-----------------------------------')
        layer_trainer.main_loop()


    encoder = DeepEncoder(layers)

    print('\n')
    print('------------------------------------------------------')
    print(' Unsupervised training done! Start Fine-tuing training...')
    print('------------------------------------------------------')
    print('\n')

    # supervised training
    #layer_trainers[-1].main_loop()
    trainer = get_layer_trainer_sgd_autoencoder(encoder,trainset[0])
    trainer.main_loop()
    return encoder,layers,trainset

def encoder_model(trainset):
    design_matrix = trainset.get_design_matrix()
    n_input = design_matrix.shape[1]
    layers = []
    structure = [[n_input, 50], [50, 10], [10, 5],[5, n_input]]
    # layer 0: gaussianRBM
    layers.append(get_grbm(structure[0]))
    # layer 1: denoising AE
    layers.append(get_grbm(structure[1]))
    # layer 2: AE
    layers.append(get_grbm(structure[2]))
    
    trainset = [ trainset ,
                TransformerDataset( raw = trainset, transformer = layers[0] ),
                TransformerDataset( raw = trainset, transformer = StackedBlocks( layers[0:2] )),
                TransformerDataset( raw = trainset, transformer = StackedBlocks( layers[0:3] ))  
                ]

    # construct layer trainers
    layer_trainers = []
    layer_trainers.append(get_layer_trainer_sgd_rbm(layers[0], trainset[0],100))
    layer_trainers.append(get_layer_trainer_sgd_rbm(layers[1], trainset[1],5))
    layer_trainers.append(get_layer_trainer_sgd_rbm(layers[2], trainset[2],5))
    #layer_trainers.append(get_layer_trainer_logistic(layers[3], trainset[3]))
    # unsupervised pretraining
    for i, layer_trainer in enumerate(layer_trainers[0:3]):
        print('-----------------------------------')
        print(' Unsupervised training layer %d, %s'%(i, layers[i].__class__))
        print('-----------------------------------')
        layer_trainer.main_loop()

    encoder = DeepEncoder(layers)

    print('\n')
    print('------------------------------------------------------')
    print(' Unsupervised training done! Start Fine-tuing training...')
    print('------------------------------------------------------')
    print('\n')
    # supervised training
    #layer_trainers[-1].main_loop()
    trainer = get_layer_trainer_sgd_autoencoder(encoder,trainset[0])
    trainer.main_loop()
    return encoder



In [47]:
Training,Testing = load_data()
encoder_models = []
for train in Training:
    encoder_models.append(encoder_model(train))

-----------------------------------
 Unsupervised training layer 0, <class 'pylearn2.models.rbm.GaussianBinaryRBM'>
-----------------------------------
Parameter and initial learning rate summary:
	W: 0.1
	bias_vis: 0.1
	bias_hid: 0.1
	sigma_driver: 0.1
Compiling sgd_update...
Compiling sgd_update done. Time elapsed: 0.552615 seconds
compiling begin_record_entry...
compiling begin_record_entry done. Time elapsed: 0.025898 seconds
Monitored channels: 
	bias_hid_max
	bias_hid_mean
	bias_hid_min
	bias_vis_max
	bias_vis_mean
	bias_vis_min
	h_max
	h_mean
	h_min
	learning_rate
	objective
	reconstruction_error
	total_seconds_last_epoch
	training_seconds_this_epoch
Compiling accum...
graph size: 83
Compiling accum done. Time elapsed: 0.548078 seconds
Monitoring step:
	Epochs seen: 0
	Batches seen: 0
	Examples seen: 0
	bias_hid_max: -2.0
	bias_hid_mean: -2.0
	bias_hid_min: -2.0
	bias_vis_max: 0.0
	bias_vis_mean: 0.0
	bias_vis_min: 0.0
	h_max: 0.984289745445
	h_mean: 0.137645287166
	h_min: 1.258

In [62]:
X = encoder_models[0].get_input_space().make_theano_batch(batch_size=47)
Y = encoder_models[0].reconstruct( X )
f1 = theano.function( [X], Y )
Y2 = encoder_models[1].reconstruct( X )
f2 = theano.function( [X], Y2 )
Y3 = encoder_models[2].reconstruct( X )
f3 = theano.function( [X], Y3 )

In [63]:
R1 = f1(testset[1].X)
R2 = f2(testset[1].X)
R3 = f3(testset[1].X)

In [75]:
((R3 - testset[1].X)**2).sum(axis=1).sum()

9535.1425419064435

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ..., 
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [80]:
encoder_models

In [35]:
a = encoder._params[1]
b = encoder._params[1]


In [31]:
import theano.tensor as T
type(T.scalar())

theano.tensor.var.TensorVariable

In [52]:
import copy
b = copy.deepcopy(a)

In [28]:
type(encoder._params[0])

theano.tensor.sharedvar.TensorSharedVariable

In [7]:
from pylearn2.utils import sharedX
a =sharedX(0,'w')

In [5]:
trainset,testset = load_data()

NameError: name 'encoder' is not defined

In [3]:
a = T.scalar()

In [39]:
import numpy as np
w = np.asarray([[1,2,3],[4,5,6],[7,8,9]])
w.sum(axis=0)

array([12, 15, 18])

0.17842313812242389